In [1]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

import neural_network_methods as nn

In [2]:
data_total = np.loadtxt('diabetes_dataset_total.csv', str)
np.version.version
# data = np.loadtxt('diabetes_dataset.csv', str)
# data_test = np.loadtxt('diabetes_test_set.csv', str)

'1.26.4'

In [3]:
def construct_X(data):
    
    n = len(data)
    num_features = 0
    
    for i in range(len(data[0])):
        if(data[0][i] == ","):
            num_features += 1
    
    X = np.zeros((n, num_features))
    for i in range(n):
        arr = data[i].split(",")
        for j in range(num_features):
            X[i][j] = arr[j]
        
    return X
    

def construct_Y(data):
    
    n = len(data)
    
    Y = np.zeros(n)
    for i in range(n):
        Y[i] = data[i][-1]
        
    return Y

def normalize(X):
    num_features = X.shape[0]
    for i in range(num_features):
        X[i] = (X[i]-np.mean(X[i]))/np.std(X[i])
        
    return X

In [4]:
def split_train_test(data, num_train):
    
    train = data[0:num_train]
    test  = data[num_train:]
    
    return train, test

In [5]:
X_total = construct_X(data_total)
Y_total = construct_Y(data_total)
print(X_total.shape)

(768, 8)


In [6]:
X_total = construct_X(data_total)
Y_total = construct_Y(data_total)

X, test_X, Y, test_Y = train_test_split(X_total, Y_total, train_size = .912, random_state = 2)
# train, test = split_train_test(data_total, 700)
print(X.shape, test_X.shape)

(700, 8) (68, 8)


In [7]:
num_features = len(X_total[1])
m = len(X)

# layer_array = [10, 1]
# activation_array = ['relu', 'sigmoid']

# layer_array = [20, 10, 5, 3, 1]
layer_array = [10, 5, 3, 1]
# layer_array = [25, 15, 10, 6, 3, 1]
# layer_array = [10, 5, 3, 1]
activation_array = ['Relu', 'Relu', 'Relu', 'sigmoid']

In [8]:
def neural_network(X, Y, layer_array, activation_array, lamb, optimizer = 'normal', num_iter=10000):
    
    L = len(layer_array)
    cost_array = np.zeros(int(num_iter/1000))
    alpha = .005
    t = 0
    beta_momentum = .95
    beta_adam_1 = .92
    beta_adam_2 = .998
    eps = 1e-8
    params = nn.initialize_params(layer_array, X, 0.07)
#     print(params)
    if(optimizer == 'momentum'):
        v = nn.initialize_optimizer(params, layer_array, optimizer)
    elif(optimizer == 'Adam'):    
        v, s = nn.initialize_optimizer(params, layer_array, optimizer)

    for i in range(num_iter):
        A, stored_Al_Zl = nn.forward_prop(params, activation_array, layer_array, X)
        
        cost = nn.comp_cost(Y, stored_Al_Zl[f"A{L}"], params, L, lamb, regularization = True)
        
        grads = nn.back_prop(layer_array, activation_array, stored_Al_Zl, params, X, Y, lamb)
        
        if(optimizer == 'normal'):
            params = nn.update_params(params, grads, alpha, layer_array, 0, 0, t, 0, 0, 0, eps, optimizer)
        elif(optimizer == 'momentum'):
            params, v = nn.update_params(params, grads, alpha, layer_array, v, 0, 0, beta_momentum, 0, 0, eps, optimizer)
        elif(optimizer == 'Adam'):
            params, v, s, v_fixed, s_fixed = nn.update_params(params, grads, alpha, layer_array, v, s, t, beta_momentum, beta_adam_1, beta_adam_2, eps, optimizer)
            t = t+1
            print(grads)
        #         params = nn.update_params(params, grads, .1, layer_array)
        
        if(i%1000 == 0):
            print(i, cost)
            index = i%1000
            cost_array[index] = cost
            
    return params, cost_array

In [9]:
print(X.shape, Y.shape)

(700, 8) (700,)


In [10]:
params, cost_array = neural_network(X.T, Y, layer_array, activation_array, 0.2, optimizer = 'momentum', num_iter=110000)

0 0.6930993014834315
1000 0.6490905071709266
2000 0.6451119165145215
3000 0.6444942008420295
4000 0.6439987194869432
5000 0.6427454078238553
6000 0.6371208543746477
7000 0.6194027986822355
8000 0.6070328496188127
9000 0.5990400125565531
10000 0.591665638949107
11000 0.5841678315037263
12000 0.575038306257653
13000 0.5666858573833473
14000 0.5578300803756271
15000 0.5454973621976403
16000 0.5311081112611415
17000 0.5151622719560203
18000 0.49638385253489165
19000 0.47795297492511735
20000 0.4622478550329726
21000 0.4487463929014251
22000 0.43395441988757577
23000 0.46429497417423715
24000 0.43765772611448306
25000 0.4363092819216793
26000 0.41879102382544253
27000 0.42727904221451474
28000 0.4074203183671921
29000 0.4452094611904327
30000 0.425763620558418
31000 0.3994185494064209
32000 0.5376021095944183
33000 0.41359803204951295
34000 0.3912279092204848
35000 0.39912544682394324
36000 0.39674402265393294
37000 0.3827250117673348
38000 0.383106210090126
39000 0.38293822804642835
40000 

In [11]:
preds_train,A_final, accuracy = nn.predict(params, activation_array, layer_array, X.T, Y)

In [12]:
print('Training set accuracy:',accuracy)

Training set accuracy: 0.8257142857142857


In [13]:
preds_test,A_final, accuracy = nn.predict(params, activation_array, layer_array, test_X.T, test_Y)
print('Test set accuracy:',accuracy)

Test set accuracy: 0.7647058823529411


In [14]:
print('F1 score:', f1_score(test_Y, preds_test[0]))
print('Precision score:', precision_score(test_Y, np.int64(preds_test)[0]))
print('Recall score:', recall_score(test_Y, np.int64(preds_test)[0]))

F1 score: 0.68
Precision score: 0.7083333333333334
Recall score: 0.6538461538461539


Selected Cases (700 train, 68 test): 

       Layers | Initial Factor|beta_1| alpha | lambda | train | test  | f1    | precision | recall| layer breakdown| Runs
       5      |   0.07        |0.00  | .1    | 0.2    | .752  | .721  | .5366 | .7333     | .4231 |  (20,10,5,3,1) | 20000
       5      |   0.07        |0.00  | .025  | 0.5    | .867  | .750  | .6909 | .6552     | .7308 |  (20,10,5,3,1) | 50000
       5      |   0.07        |0.00  | .010  | 0.5    | .849  | .750  | .7213 | .6286     | .8462 |  (20,10,5,3,1) | 50000
       
       **Momentum**
       5      |   0.07        |0.91  | .1    | 0.20   | .851  | .750  | .6909 | .6552     | .7308 |  (10,7,5,3,1)  | 50000
       5      |   0.07        |0.95  | .1    | 0.20   | .909  | .794  | .7407 | .7143     | .7692 |  (20,10,5,3,1) | 20000
       5      |   0.07        |0.96  | .1    | 0.20   | .959  | .662  | .5818 | .5517     | .6154 |  (20,10,5,3,1) | 20000
       5      |   0.07        |0.97  | .1    | 0.15   | .904  | .765  | .6923 | .6923     | .6923 |  (20,10,5,3,1) | 20000
       4      |   0.07        |0.95  | .075  | 0.20   | .837  | .779  | .7059 | .7200     | .6923 |  (10,5,3,1)    | 110000
       4      |   0.07        |0.90  | .005  | 0.20   | .877  | .779  | .7368 | .6774     | .8077 |  (15,5,3,1)    | 110000
       4      |   0.07        |0.95  | .005  | 0.20   | .904  | .750  | .6909 | .6552     | .7308 |  (17,5,3,1)    | 110000

Special thanks to DeepLearning.ai for the coursework/syntax help on the NN structure codes.

Retrieved from DeepLearning.AI (2024, December 25). 

DeepLearning.AI (December 2024). Logistic Regression with a Neural Network Mindset. Retrieved from C1_W2 Graded Assignment 1 in the Deep Learning Specialization

DeepLearning.AI (December 2024). Planar Data Classification with One Hidden layer. Retrieved from C1_W3 Graded Assignment 1 in the Deep Learning Specialization

DeepLearning.AI (December 2024). Build your Deep Neural Network: Step by Step. Retrieved from C1_W4 Graded Assignment 1 in the Deep Learning Specialization

DeepLearning.AI (December 2024). Deep Neural Network - Application. Retrieved from C1_W4 Graded Assignment 3 in the Deep Learning Specialization

DeepLearning.AI (December 2024). Gradient Checking. Retrieved from C2_W1 Graded Assignment 3 in the Deep Learning Specialization

DeepLearning.AI (December 2024). Optimization methods. Retrieved from C2_W2 Graded Assignment 1 in the Deep Learning Specialization